# Train and Deploy Your First Machine Learning Model on Amazon SageMaker

## Create SageMaker session

A SageMaker session needs to be initialized in order to start interacting the SageMaker service.

In [15]:
import boto3
import re

import os
import numpy as np
import pandas as pd
import sagemaker as sage
import json
import pickle
import sys
import traceback

from sklearn import ensemble
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split

boto_session = boto3.Session(profile_name="up-sagemaker")
session = sage.Session(boto_session=boto_session)

## Train and save the model locally

In [16]:
def _save_model(clf):
    with open(os.path.join("model/", 'model.pkl'), 'wb') as out:
        pickle.dump(clf, out)
        
def train():
    print('Starting the training.')
    try:
        data_df = pd.read_csv("data/iris.csv", header=None)

        # labels are in the first column
        labels = data_df.iloc[:, 0]
        features = data_df.iloc[:, 1:]

        train_features, test_features, labels_train, labels_test = \
            train_test_split(
                features, labels, test_size=0.3, random_state=42
            )

        max_leaf_nodes = 100

        clf = ensemble.RandomForestClassifier(
            max_leaf_nodes=max_leaf_nodes
        )
        clf = clf.fit(train_features, labels_train)

        test_predictions = clf.predict(test_features)

        precision = precision_score(
            labels_test, test_predictions, average='macro'
        )
        recall = recall_score(
            labels_test, test_predictions, average='macro'
        )

        clf = clf.fit(features, labels)
        _save_model(clf)

        print('Training complete.')
    except Exception as e:
        print(
            'An Exception during training: ' + str(e) + '\n'
        )
train()

Starting the training.
Training complete.


## Upload the model to S3 bucket 

Using the SageMaker session, which was initialized earlier, the model will be upload to S3. 

In [17]:
local_model_directory = 'model'
s3_prefix = 'up-lambda-iris-model'

model_location = session.upload_data(local_model_directory, key_prefix=s3_prefix)

## Deploy the model

Using the trained `Estimator`, a RESTful service will be initialized on Amazon SageMaker service.

In [8]:
from sagemaker.predictor import json_serializer

predictor = estimator.deploy(1, 'ml.m4.xlarge', serializer=json_serializer)

-----------!

## Sample some data and use it for a prediction

In [5]:
data_df = pd.read_csv("data/iris.csv", header=None, names=["feat_1", "feat_2", "feat_3", "feat_4"])

sampled_df = data_df.sample(10)

sampled_post_request_body = sampled_df.to_dict(orient='list')

sampled_post_request_body

{'feat_1': [5.8, 5.4, 5.0, 6.4, 5.7, 5.8, 5.4, 4.8, 5.0, 5.5],
 'feat_2': [2.7, 3.9, 3.4, 3.2, 2.5, 2.6, 3.9, 3.1, 3.0, 4.2],
 'feat_3': [3.9, 1.7, 1.6, 5.3, 5.0, 4.0, 1.3, 1.6, 1.6, 1.4],
 'feat_4': [1.2, 0.4, 0.4, 2.3, 2.0, 1.2, 0.4, 0.2, 0.2, 0.2]}

In [6]:
print(predictor.predict(sampled_post_request_body).decode('utf-8'))

{"predictions": ["versicolor", "setosa", "setosa", "virginica", "virginica", "versicolor", "setosa", "setosa", "setosa", "setosa"]}


## Delete the endpoint

In [7]:
session.delete_endpoint(predictor.endpoint)